In [22]:
# import autoreload
# ?autoreload

%load_ext autoreload
%autoreload 2
import torch
import time
from transformers import AutoModel
from transformers import AutoConfig

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from calibrate_e_ml import calibrate_e_ml
from cg.node import construct_aggregation_graph

In [32]:
seq_len = 100
bs = 4
input_ids = torch.randint(1000, size=(bs, seq_len), dtype=torch.long, device='cpu')
model_name = "prajjwal1/bert-tiny"
cuda_exist = torch.cuda.is_available()
device = torch.device("cuda" if cuda_exist else "cpu")
information = calibrate_e_ml(model_name, bs, seq_len, device)

In [ ]:
input_ids.dtype

In [ ]:
information

In [25]:
level = information['linear'][0]
print(level)
fn = level['module']
fname=level['name']
fi_size, fi_dtype = level['inputs']
fo_size, fo_dtype = level['outputs']
fi = torch.rand(fi_size, dtype=fi_dtype)
fo = torch.rand(fo_size, dtype=fo_dtype)

fo_call = fn(fi)

{'name': 'encoder.layer.0.attention.self.query:Linear', 'module': Linear(in_features=128, out_features=128, bias=True), 'inputs': (torch.Size([4, 100, 128]), torch.float32), 'outputs': (torch.Size([4, 100, 128]), torch.float32), 'runtime': 0.012598949000000914}


In [26]:
f_start = time.perf_counter()
for _ in range(10000):
    fo_call = fn(fi)
f_end = time.perf_counter()
print(f_end-f_start)

4.047626644999809


In [27]:
trace = torch.jit.trace(fn, fi)
trace_graph = trace.inlined_graph
trace_graph

graph(%self : __torch__.torch.nn.modules.linear.___torch_mangle_1.Linear,
      %input : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu)):
  %2 : Tensor = prim::GetAttr[name="bias"](%self)
  %3 : Tensor = prim::GetAttr[name="weight"](%self)
  %4 : Float(128:1, 128:128, requires_grad=1, device=cpu) = aten::t(%3) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1692:0
  %output : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu) = aten::matmul(%input, %4) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1692:0
  %6 : int = prim::Constant[value=1]() # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1694:0
  %m_out : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu) = aten::add_(%output, %2, %6) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1694:0
  return (%m_out)

In [28]:
graph, op_data_types = construct_aggregation_graph(trace_graph, fname)

/Users/qqcao/work/DetecTor/cg/op_counter.py:127: UserWarning: no counter available for prim::GetAttr!
  else:
/Users/qqcao/work/DetecTor/cg/op_counter.py:127: UserWarning: no counter available for aten::t!
  else:
/Users/qqcao/work/DetecTor/cg/op_counter.py:127: UserWarning: no counter available for prim::Constant!
  else:


In [29]:
graph

Graph(name=encoder.layer.0.attention.self.query:Linear,
	inputs=[
		DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_1.Linear)
		DataNode(id=input, dtype=Float, shape=[4, 100, 128])],
	nodes=[
		OpNode(id=2, op=prim::GetAttr, 
			inputs=[
				DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_1.Linear)],
			outputs=[
				DataNode(id=2, dtype=None)],)
		OpNode(id=3, op=prim::GetAttr, 
			inputs=[
				DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_1.Linear)],
			outputs=[
				DataNode(id=3, dtype=None)],)
		OpNode(id=4, op=aten::t, 
			inputs=[
				DataNode(id=3, dtype=None)],
			outputs=[
				DataNode(id=4, dtype=Float, shape=[128, 128])],)
		OpNode(id=output, op=aten::matmul, flops=13107200, mem_bytes=671744, 
			inputs=[
				DataNode(id=input, dtype=Float, shape=[4, 100, 128])
				DataNode(id=4, dtype=Float, shape=[128, 128])],
			outputs=[
				DataNode(id=output, dtype=Float, shape=[4, 100, 128])],)
		OpNode(i

```
for each input size (bs, seq_len) pair:
    for each module:
        get jit trace, extract flops and mem_bytes,
        log (bs, seq_len, name, module level, flops, mem_bytes)
        run the module 100k times, log timestamp
        sleep 5s
        repeat 10 rounds
record energy with timestamp, log cpu_util, gpu_util, mem_util, gpu_util

extract data (bs, seq_len, name, module level, flops, mem_bytes, cpu_util, gpu_util, mem_util, gpu_util, energy)
calc energy variance, simple linear regression fitting
```   

In [30]:
from collections import defaultdict

def get_flops_mem_bytes(graph):
    flops = dict()
    mem_bytes = dict()

    for node in graph.nodes:
        flops[node.id] = node.flops
        mem_bytes[node.id] = node.mem_bytes
        if node.flops:
            print(node.op, node.flops, node.mem_bytes)
    return sum(flops.values()), sum(mem_bytes.values())

In [31]:
get_flops_mem_bytes(graph)

aten::matmul 13107200 671744
aten::add_ 51200 409600


(13158400, 1081344)

In [ ]:
from datetime import datetime
print(datetime.fromtimestamp(1605498402.136237))

In [118]:
import pandas
pandas.set_option('display.float_format', '{:.6f}'.format)

res= pandas.read_csv('data/mlexp/mlexp-linear-res2.csv')
energy= pandas.read_csv('data/mlexp/mlexp-linear-energy2.csv', error_bad_lines=False, usecols=[0, 2])

In [88]:
import json

model_name = 'bert-base-uncased'
level = 'linear'
runs = 10
bs = 16
seq_len = 384

prof_filename = f'{model_name}_{level}_r{runs}_b{bs}_i{seq_len}.json'

with open(f'data/mlexp/{prof_filename}') as f:
    prof_info = json.load(f)

In [89]:
prof_info[0]

{'name': 'encoder.layer.0.attention.self.query:Linear',
 'flops': 7252475904,
 'mem_bytes': 113246208,
 'start_1': 1610903590.6697361,
 'end_1': 1610903591.3599648,
 'start_2': 1610903592.3611808,
 'end_2': 1610903593.052057,
 'start_3': 1610903594.0532901,
 'end_3': 1610903594.7442207,
 'start_4': 1610903595.7448068,
 'end_4': 1610903596.435857,
 'start_5': 1610903597.4370837,
 'end_5': 1610903598.1282494,
 'start_6': 1610903599.1294367,
 'end_6': 1610903599.824273,
 'start_7': 1610903600.8254433,
 'end_7': 1610903601.5203035,
 'start_8': 1610903602.5215309,
 'end_8': 1610903603.216559,
 'start_9': 1610903604.2176173,
 'end_9': 1610903604.9122813,
 'start_10': 1610903605.913508,
 'end_10': 1610903606.6082392,
 'type': 'linear'}

In [119]:
def is_float(x):
    try:
        float(x)
    except ValueError:
        return False
    return True

energy=energy[energy['timestamp'].apply(lambda x: is_float(x))]
energy['value']=energy['value'].div(100)
energy['timestamp']=energy['timestamp'].astype(float)

In [129]:
res_np = res.to_numpy()
energy_np = energy.to_numpy()

In [131]:
res

,timestamp,cpu,mem,n_gpu,gpu,gpu_mem,gpu_power,gpu.1,gpu_mem.1,gpu_power.1
0,1610903583.440226,2.600000,4.000000,2,0,0,15313,2,0,34326
1,1610903583.540479,1.800000,4.000000,2,0,0,15313,0,0,34343
2,1610903583.640747,0.100000,4.000000,2,0,0,15313,0,0,34491
3,1610903583.741015,0.100000,4.000000,2,0,0,15216,0,0,34587
4,1610903583.841277,0.100000,4.000000,2,0,0,15313,0,0,34491
...,...,...,...,...,...,...,...,...,...,...
1684,1610903752.278331,0.100000,4.000000,2,0,0,10150,0,0,6495
1685,1610903752.378598,0.100000,4.000000,2,0,0,10059,0,0,6400
1686,1610903752.478866,0.100000,4.000000,2,0,0,9861,0,0,6592
1687,1610903752.579133,0.100000,4.000000,2,0,0,10052,0,0,6397


In [132]:
res_t = res_np[:, 0]
energy_t = energy_np[:, 0]

In [133]:
energy_np

array([[1.61090358e+09, 1.12330000e+02],
       [1.61090358e+09,            nan],
       [1.61090358e+09, 1.00000000e-02],
       ...,
       [1.61090376e+09, 1.15410000e+02],
       [1.61090376e+09, 1.15140000e+02],
       [1.61090376e+09, 1.13470000e+02]])

In [134]:
import bisect

res_s = bisect.bisect_right(res_t, prof_info[0]['start_1'])
res_e = bisect.bisect_right(res_t, prof_info[0]['end_1'])

energy_s = bisect.bisect_right(energy_t, prof_info[0]['start_1'])
energy_e = bisect.bisect_right(energy_t, prof_info[0]['end_1'])

In [138]:
res[res_s:res_e]

,timestamp,cpu,mem,n_gpu,gpu,gpu_mem,gpu_power,gpu.1,gpu_mem.1,gpu_power.1
73,1610903590.760744,9.300000,9.000000,2,99,50,208903,0,0,34360
74,1610903590.861005,8.900000,9.000000,2,99,50,212446,0,0,34502
75,1610903590.961265,10.400000,9.000000,2,100,39,210800,0,0,34502
76,1610903591.061532,9.200000,9.000000,2,100,39,212381,0,0,34511
77,1610903591.161789,8.400000,9.000000,2,100,39,211546,0,0,34502
78,1610903591.262048,8.400000,9.000000,2,100,39,211469,0,0,34502


In [215]:
res

,timestamp,cpu,mem,n_gpu,gpu,gpu_mem,gpu_power,gpu.1,gpu_mem.1,gpu_power.1
0,1610914901.591612,1.100000,4.000000,2,0,0,15846,0,0,34087
1,1610914901.761873,0.600000,4.000000,2,0,0,15751,0,0,34135
2,1610914901.932141,0.100000,4.000000,2,0,0,15749,0,0,34149
3,1610914902.102404,0.100000,4.000000,2,0,0,15312,0,0,34149
4,1610914902.272667,0.100000,4.000000,2,0,0,15654,0,0,34246
...,...,...,...,...,...,...,...,...,...,...
64732,1610926947.577797,8.200000,9.000000,2,100,50,211232,0,0,6603
64733,1610926947.748063,8.100000,9.000000,2,100,51,213543,0,0,6603
64734,1610926947.918305,8.200000,9.000000,2,100,50,210710,0,0,6603
64735,1610926948.088569,8.100000,9.000000,2,100,50,214403,0,0,6698


In [147]:
import numpy as np

for prof_item in prof_info:
    energy_runs = []
    for r in range(1, runs+1):
        energy_s = bisect.bisect_right(energy_t, prof_item[f'start_{r}'])
        energy_e = bisect.bisect_right(energy_t, prof_item[f'end_{r}'])
        energy_r = energy[energy_s:energy_e]['value'].sum()
        energy_runs.append(energy_r)
    std_percent = np.std(energy_runs) / np.mean(energy_runs)
    mean = np.mean(energy_runs)
    print(prof_item['type'], prof_item['name'], std_percent, mean)

linear encoder.layer.0.attention.self.query:Linear 0.07236582207171405 1242.308
linear encoder.layer.0.intermediate.dense:Linear 0.13399399857273184 8558.122
linear encoder.layer.0.output.dense:Linear 0.12190375616305438 7355.465000000001
linear pooler.dense:Linear 0.8961761566460925 93.628


In [217]:
import bisect
import json
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option('display.float_format', '{:.6f}'.format)
def is_float(x):
    try:
        float(x)
    except ValueError:
        return False
    return True

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args=Namespace(out_dir='data', runs=10, batch_size=34, batch_step=2, seq_step=16, input_length=400)

model_name = 'bert-base-uncased'
level = 'level'

out_dir = Path(args.out_dir)
out_dir.mkdir(parents=True, exist_ok=True)
runs = args.runs
input_length = args.input_length
batch_size = args.batch_size
batch_step = args.batch_step
seq_step = args.seq_step


In [232]:
feature_file = out_dir / f'{model_name}_features.csv'

res = pd.read_csv(out_dir / f'res-{model_name}.csv')
energy = pd.read_csv(out_dir / f'energy-{model_name}.csv',
                     error_bad_lines=False, usecols=[0, 2])
energy = energy[energy['value'].apply(lambda x: is_float(x))]
energy = energy[energy['timestamp'].apply(lambda x: is_float(x))]

energy['value'] = energy['value'].astype(float).div(100)
energy['timestamp'] = energy['timestamp'].astype(float)

res_np = res.to_numpy()
res_t = res_np[:, 0]
energy_np = energy.to_numpy()
energy_t = energy_np[:, 0]
res_names = ['cpu', 'mem', 'gpu', 'gpu_mem']

feature_names = ['batch_size', 'seq_len', 'flops',
                 'mem_bytes'] + res_names + [f'{k}_std'
                                             for k in res_names] + \
                ['times_mean', 'times_std',
                 'gpu_power_mean', 'gpu_power_std',
                 'energy_mean', 'energy_std',
                 'level_name', 'model_name']
feature_values = {k: [] for k in feature_names}

for bs in list(range(2, batch_size, batch_step)) + [1]:
    for seq_len in range(16, input_length, seq_step):
        filename = f'{model_name}_{level}_r{runs}_b{bs}_i{seq_len}.json'
        prof_file = Path(out_dir) / 'mlexp' / filename
        if not prof_file.exists():
            continue
        with open(prof_file) as f:
            prof_info = json.load(f)

        for prof_item in prof_info:
            gpu_power_runs = []
            energy_runs = []
            times_runs = []
            res_runs = {k: [] for k in res_names}
            repeats = prof_item['repeats']
            for r in range(1, runs + 1):
                start_r = prof_item[f'start_{r}']
                end_r = prof_item[f'end_{r}']
                times_runs.append(end_r - start_r)

                res_s = bisect.bisect_right(res_t, start_r)
                res_e = bisect.bisect_right(res_t, end_r)
                res_r = res[res_s:res_e]
                for rn in res_names:
                    res_runs[rn].append(res_r[rn].mean())
                gpu_power_r = res_r['gpu_power'].sum()
                gpu_power_runs.append(gpu_power_r)

                e_s = bisect.bisect_right(energy_t, start_r)
                e_e = bisect.bisect_right(energy_t, end_r)
                energy_r = energy[e_s:e_e]['value'].sum().div(repeats)
                energy_runs.append(energy_r)

            times_mean = np.mean(times_runs)
            times_std = np.std(times_runs) / times_mean * 100
            gpu_power_mean = np.mean(gpu_power_runs)
            gpu_power_std = np.std(gpu_power_runs) / gpu_power_mean * 100
            energy_mean = np.mean(energy_runs)
            energy_std = np.std(energy_runs) / energy_mean * 100
            for rn in res_names:
                feature_values[rn].append(np.mean(res_runs[rn]))
                rn_std = np.std(res_runs[rn]) / np.mean(res_runs[rn]) * 100
                feature_values[f'{rn}_std'].append(rn_std)

            flops = prof_item['flops'] / 1e6
            mem_bytes = prof_item['mem_bytes'] / 1024 / 1024
            feature_values['batch_size'].append(bs)
            feature_values['seq_len'].append(seq_len)
            feature_values['energy_mean'].append(energy_mean)
            feature_values['energy_std'].append(energy_std)
            feature_values['gpu_power_mean'].append(gpu_power_mean)
            feature_values['gpu_power_std'].append(gpu_power_std)
            feature_values['flops'].append(flops)
            feature_values['mem_bytes'].append(mem_bytes)
            feature_values['times_mean'].append(times_mean)
            feature_values['times_std'].append(times_std)
            feature_values['level_name'].append(prof_item['name'])
            feature_values['model_name'].append(model_name)

            print(f"b{bs},i{seq_len}:{prof_item['type']},"
                  f" {energy_std:.1f}%, {energy_mean / 10:.2f} J,"
                  f" {flops:.2f} MFlops,"
                  f" {mem_bytes:.2f} MiB,"
                  f" {prof_item['name']}")
info = pd.DataFrame(data=feature_values)

b32,i336:linear, 0.1%, 4514.13 J, 12691.83 MFlops, 198.00 MiB, encoder.layer.0.attention.self.query:Linear
b32,i336:linear, 1.3%, 19155.45 J, 50767.33 MFlops, 697.50 MiB, encoder.layer.0.intermediate.dense:Linear
b32,i336:linear, 1.1%, 16598.96 J, 50742.56 MFlops, 508.50 MiB, encoder.layer.0.output.dense:Linear
b32,i336:linear, 14.5%, 66.40 J, 37.75 MFlops, 2.44 MiB, pooler.dense:Linear
b32,i352:linear, 0.4%, 4733.11 J, 13296.21 MFlops, 204.00 MiB, encoder.layer.0.attention.self.query:Linear
b32,i352:linear, 1.4%, 19866.67 J, 53184.82 MFlops, 717.00 MiB, encoder.layer.0.intermediate.dense:Linear
b32,i352:linear, 1.2%, 17291.65 J, 53158.87 MFlops, 519.00 MiB, encoder.layer.0.output.dense:Linear
b32,i352:linear, 15.8%, 61.55 J, 37.75 MFlops, 2.44 MiB, pooler.dense:Linear
b32,i368:linear, 0.7%, 4913.31 J, 13900.58 MFlops, 210.00 MiB, encoder.layer.0.attention.self.query:Linear
b32,i368:linear, 1.4%, 20655.59 J, 55602.32 MFlops, 736.50 MiB, encoder.layer.0.intermediate.dense:Linear
b32,i36

In [233]:
for k, v in feature_values.items():
    print(k, len(v))

batch_size 44
seq_len 44
flops 44
mem_bytes 44
cpu 44
mem 44
gpu 44
gpu_mem 44
cpu_std 44
mem_std 44
gpu_std 44
gpu_mem_std 44
times_mean 44
times_std 44
gpu_power_mean 44
gpu_power_std 44
energy_mean 44
energy_std 44
level_name 44
model_name 44


In [234]:
pd.set_option('display.max_rows', 300)
# res

In [235]:
info.sort_values('energy_std')

,batch_size,seq_len,flops,mem_bytes,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name
0,32,336,12691.832832,207.618048,8.499719,9.000000,99.999270,40.842884,0.606985,0.000000,0.002190,0.188375,23.773099,0.534546,29661294.900000,0.202801,45141.313000,0.143820,encoder.layer.0.attention.self.query:Linear,bert-base-uncased
4,32,352,13296.205824,213.909504,8.525396,9.000000,99.999310,41.213566,0.528170,0.000000,0.002069,0.267621,25.123044,0.725997,30982004.800000,0.766983,47331.130000,0.446966,encoder.layer.0.attention.self.query:Linear,bert-base-uncased
8,32,368,13900.578816,220.200960,8.541976,9.000000,99.999338,42.510828,0.867159,0.000000,0.001987,0.088140,26.171341,0.731167,32458939.900000,0.915528,49133.136000,0.670398,encoder.layer.0.attention.self.query:Linear,bert-base-uncased
37,1,368,1737.572352,24.133632,8.371039,9.000000,95.106494,33.326190,1.203322,0.000000,1.773190,1.778730,3.700568,0.016697,4188442.500000,1.712958,6368.135000,1.107965,encoder.layer.0.intermediate.dense:Linear,bert-base-uncased
2,32,336,50742.558720,533.200896,8.516921,9.000000,99.999793,28.488300,0.105433,0.000000,0.000620,0.198788,85.442235,1.211194,109914095.300000,1.215346,165989.615000,1.136470,encoder.layer.0.output.dense:Linear,bert-base-uncased
6,32,352,53158.871040,544.210944,9.230134,9.000000,99.999805,29.276063,0.201126,0.000000,0.000585,0.357797,90.543401,1.218499,114162739.100000,1.196775,172916.481000,1.220853,encoder.layer.0.output.dense:Linear,bert-base-uncased
12,32,384,14504.951808,226.492416,8.550569,9.000000,99.999363,40.041305,1.398983,0.000000,0.001911,0.102410,27.396734,0.824995,34372119.100000,1.411255,51733.005000,1.241507,encoder.layer.0.attention.self.query:Linear,bert-base-uncased
1,32,336,50767.331328,731.381760,8.520592,9.000000,99.999825,49.459245,0.175612,0.000000,0.000525,0.049609,101.119955,1.258644,125477770.500000,1.278958,191554.458000,1.288297,encoder.layer.0.intermediate.dense:Linear,bert-base-uncased
10,32,368,55575.183360,555.220992,9.107070,9.000000,99.999813,29.252714,0.147388,0.000000,0.000561,0.199918,94.590829,1.259905,119163525.900000,1.254162,180166.952000,1.295936,encoder.layer.0.output.dense:Linear,bert-base-uncased
14,32,384,57991.495680,566.231040,8.631334,9.000000,99.999822,27.477014,0.173255,0.000000,0.000533,0.274166,99.684819,1.243543,124748070.300000,1.248362,189365.937000,1.329011,encoder.layer.0.output.dense:Linear,bert-base-uncased


In [237]:
info.sort_values('times_mean', ascending=False)

,batch_size,seq_len,flops,mem_bytes,cpu,mem,gpu,gpu_mem,cpu_std,mem_std,gpu_std,gpu_mem_std,times_mean,times_std,gpu_power_mean,gpu_power_std,energy_mean,energy_std,level_name,model_name
13,32,384,58019.807232,792.723456,9.381419,9.000000,99.999847,49.587872,2.608006,0.000000,0.000460,0.057627,115.472171,1.297657,147342731.000000,1.443132,223366.437000,1.575258,encoder.layer.0.intermediate.dense:Linear,bert-base-uncased
9,32,368,55602.315264,772.276224,8.533256,9.000000,99.999196,50.007333,0.223738,0.000000,0.002412,0.034763,110.198429,1.280767,135510619.100000,1.321842,206555.878000,1.362068,encoder.layer.0.intermediate.dense:Linear,bert-base-uncased
5,32,352,53184.823296,751.828992,8.527498,9.000000,99.999832,49.840051,0.172494,0.000000,0.000503,0.033076,105.719738,1.269730,129924049.200000,1.286952,198666.739000,1.425890,encoder.layer.0.intermediate.dense:Linear,bert-base-uncased
1,32,336,50767.331328,731.381760,8.520592,9.000000,99.999825,49.459245,0.175612,0.000000,0.000525,0.049609,101.119955,1.258644,125477770.500000,1.278958,191554.458000,1.288297,encoder.layer.0.intermediate.dense:Linear,bert-base-uncased
14,32,384,57991.495680,566.231040,8.631334,9.000000,99.999822,27.477014,0.173255,0.000000,0.000533,0.274166,99.684819,1.243543,124748070.300000,1.248362,189365.937000,1.329011,encoder.layer.0.output.dense:Linear,bert-base-uncased
10,32,368,55575.183360,555.220992,9.107070,9.000000,99.999813,29.252714,0.147388,0.000000,0.000561,0.199918,94.590829,1.259905,119163525.900000,1.254162,180166.952000,1.295936,encoder.layer.0.output.dense:Linear,bert-base-uncased
6,32,352,53158.871040,544.210944,9.230134,9.000000,99.999805,29.276063,0.201126,0.000000,0.000585,0.357797,90.543401,1.218499,114162739.100000,1.196775,172916.481000,1.220853,encoder.layer.0.output.dense:Linear,bert-base-uncased
2,32,336,50742.558720,533.200896,8.516921,9.000000,99.999793,28.488300,0.105433,0.000000,0.000620,0.198788,85.442235,1.211194,109914095.300000,1.215346,165989.615000,1.136470,encoder.layer.0.output.dense:Linear,bert-base-uncased
12,32,384,14504.951808,226.492416,8.550569,9.000000,99.999363,40.041305,1.398983,0.000000,0.001911,0.102410,27.396734,0.824995,34372119.100000,1.411255,51733.005000,1.241507,encoder.layer.0.attention.self.query:Linear,bert-base-uncased
8,32,368,13900.578816,220.200960,8.541976,9.000000,99.999338,42.510828,0.867159,0.000000,0.001987,0.088140,26.171341,0.731167,32458939.900000,0.915528,49133.136000,0.670398,encoder.layer.0.attention.self.query:Linear,bert-base-uncased


In [247]:
int(10000 // (11648 / 1e9))

858516483

In [239]:
5074255872 // 1e9

5.0